In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import os
import tensorflow as tf
print(os.getcwd())
data_path = os.path.join(os.getcwd(), 'data')
X = np.load(os.path.join(data_path, 'X_69_200.npy'), allow_pickle=True)
# reshape X to be [n][f][t] f is frequency, t is time of the stft
freq_bins = X[0].shape[0]
data_size = X.shape[0]
# X = np.reshape(X, newshape=(data_size, nfft_bins,None ))
print(data_size, freq_bins)
Y = np.load(os.path.join(data_path, 'Y_69_200.npy'), allow_pickle=True)
# create tensor shape (data_size, nfft_bins, None) for X

# X = tf.ragged.constant(X)
# Y = tf.ragged.constant(Y)
# [print(x.shape, x.dtype, type(x)) for x in X]


In [ ]:

X = [tf.RaggedTensor.from_tensor(x, ragged_rank=1) for x in X]
Y = [tf.RaggedTensor.from_tensor(y, ragged_rank=1) for y in Y]
splits = train_test_split(X, Y, test_size=0.2, random_state=69)
# test = [tf.RaggedTensor.from_uniform_row_length(x, uniform_row_length=freq_bins).flat_values for x in splits[0]]

X_train, X_test, y_train, y_test = splits
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [33]:
def gen():
    for idx in range(X.shape[0]):
        x = X[idx]
        y = Y[idx]
        x_ragged = tf.ragged.stack(x.tolist(), axis=0)
        y_ragged = tf.ragged.stack(y.tolist(), axis=0)
        # print(x_ragged.shape, x_ragged.dtype, type(x_ragged))
        yield x_ragged, y_ragged
dataset = tf.data.Dataset.from_generator(
    generator=gen,
     
    output_signature=(
        tf.TensorSpec(shape=(513, None), dtype=np.complex64), 
        tf.TensorSpec(shape=(513, None), dtype=np.complex64)))
# dataset.batch(128).repeat()
dataset.element_spec
res = dataset.(1)
res[0]

TypeError: 'TakeDataset' object is not subscriptable

In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalizationV2, GlobalMaxPooling2D
from keras.layers import Flatten, Dropout, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import adam_v2
from keras import layers
import keras

# import rms prop from keras
# A Keras tensor is a symbolic tensor-like object, which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
opt = adam_v2.Adam(learning_rate=0.0007)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

# inputs are 2d arrays of shape [nfft_bins, time], time is a varried
lookahead = 1
frequency_bins = X[0].shape[0]
data_shape = (frequency_bins, None)
model_input = layers.Input(shape=(frequency_bins, None), dtype=tf.complex64)
print(model_input.shape)
# add channel dimension
x_imag = tf.math.imag(model_input)
# print(x_imag.shape)
x = tf.abs(model_input)
x = tf.expand_dims(x, axis=-1)
x = layers.Conv2D(filters=16, kernel_size=(frequency_bins//4,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=32, kernel_size=(frequency_bins//2,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=64, kernel_size=(1,1), padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=32, kernel_size=(frequency_bins//2,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=16, kernel_size=(frequency_bins//4,1),  padding='same', activation='relu')(x)
x = layers.Conv2D(filters=1, kernel_size=(1,1),  padding='same', activation='relu')(x)
# x = layers.GlobalMaxPooling2D(keepdims=True)(x)
# out = Dense(frequency_bins, activation="relu")(x)
x = tf.squeeze(x, axis=-1)
out = tf.complex(x, x_imag)
model = keras.Model(inputs=model_input, outputs=out)
model.summary()
model.compile(loss='mse',
              optimizer=opt, metrics=['accuracy'])

# model.add(Dropout(0.2))


# model.add(Dropout(0.2))

# model.add(Dropout(0.2))


# model.add(BatchNormalizationV2())

# model.add(GlobalMaxPooling2D())
# model.add(Flatten())



In [ ]:
dataset.shuffle(1000).batch(128)
cnnhistory=model.fit(dataset, epochs=200, callbacks=[es], verbose=2, steps_per_epoch=100, bat)
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

AttributeError: 'generator' object has no attribute 'one_shot_iterator'